In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from keras.models import Model, load_model
from keras.preprocessing import image
import time
import pickle
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.layers import *

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
import pandas as pd
dataset = pd.read_csv('/content/drive/My Drive/AI/Projects/Image captioning/Dataset/FlickrText/New folder/Flickr8k.token.txt',delimiter='\t',header=None)


In [ ]:
print(dataset.shape)

(40460, 2)


In [ ]:
dataset.head()

,0,1
0,1000268201_693b08cb0e.jpg#0,A child in a pink dress is climbing up a set o...
1,1000268201_693b08cb0e.jpg#1,A girl going into a wooden building .
2,1000268201_693b08cb0e.jpg#2,A little girl climbing into a wooden playhouse .
3,1000268201_693b08cb0e.jpg#3,A little girl climbing the stairs to her playh...
4,1000268201_693b08cb0e.jpg#4,A little girl in a pink dress going into a woo...


In [ ]:
print(dataset[0][3])
print(dataset[1][3])

1000268201_693b08cb0e.jpg#3
A little girl climbing the stairs to her playhouse .


In [ ]:
import re
import nltk


corpus = []
for i in range(0, dataset.shape[0]):
    cap = re.sub('[^a-zA-Z]', ' ', dataset[1][i])  #Removed all other characters except alphabets
    cap = cap.lower()                              #Converted to lower case
    cap = cap.split()                               
    cap=[word for word in cap if len(word)>1]      #Removed single letter words
    cap = ' '.join(cap)                            # Joined with spaces
    cap= '<startseq> '+cap+' <endseq>'
    corpus.append(cap)                             #list of captions

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
for i in range(50): print(corpus[i:i+1])
print(len(corpus))

['<startseq> child in pink dress is climbing up set of stairs in an entry way <endseq>']
['<startseq> girl going into wooden building <endseq>']
['<startseq> little girl climbing into wooden playhouse <endseq>']
['<startseq> little girl climbing the stairs to her playhouse <endseq>']
['<startseq> little girl in pink dress going into wooden cabin <endseq>']
['<startseq> black dog and spotted dog are fighting <endseq>']
['<startseq> black dog and tri colored dog playing with each other on the road <endseq>']
['<startseq> black dog and white dog with brown spots are staring at each other in the street <endseq>']
['<startseq> two dogs of different breeds looking at each other on the road <endseq>']
['<startseq> two dogs on pavement moving toward each other <endseq>']
['<startseq> little girl covered in paint sits in front of painted rainbow with her hands in bowl <endseq>']
['<startseq> little girl is sitting in front of large painted rainbow <endseq>']
['<startseq> small girl in the grass

In [ ]:
i=0
while i < dataset.shape[0]:
  dataset[0][i]=dataset[0][i].split('.')[0]        #Removed characters after '.'
  i=i+1

In [ ]:
print(dataset.head)

<bound method NDFrame.head of                            0                                                  1
0      1000268201_693b08cb0e  A child in a pink dress is climbing up a set o...
1      1000268201_693b08cb0e              A girl going into a wooden building .
2      1000268201_693b08cb0e   A little girl climbing into a wooden playhouse .
3      1000268201_693b08cb0e  A little girl climbing the stairs to her playh...
4      1000268201_693b08cb0e  A little girl in a pink dress going into a woo...
...                      ...                                                ...
40455   997722733_0cb5439472           A man in a pink shirt climbs a rock face
40456   997722733_0cb5439472           A man is rock climbing high in the air .
40457   997722733_0cb5439472  A person in a red shirt climbing up a rock fac...
40458   997722733_0cb5439472                    A rock climber in a red shirt .
40459   997722733_0cb5439472  A rock climber practices on a rock climbing wa...

[40460 ro

In [ ]:
type(dataset)

pandas.core.frame.DataFrame

In [ ]:
from collections import defaultdict
new_dict= defaultdict(lambda : [])          #Created a dictionary with image ids as key and captions as value
i=0
while i <len(corpus):
  # print(dataset[0][i])
  new_dict[dataset[0][i]].append(corpus[i])
  i=i+1                                                     

In [ ]:
print(len(new_dict))

8092


In [ ]:
print(new_dict['2513260012_03d33305cf'])

['<startseq> black dog is running after white dog in the snow <endseq>', '<startseq> black dog chasing brown dog through snow <endseq>', '<startseq> two dogs chase each other across the snowy ground <endseq>', '<startseq> two dogs play together in the snow <endseq>', '<startseq> two dogs running through low lying body of water <endseq>']


In [ ]:
all_vocab = []              #A list of all the words in the captions

for key in new_dict.keys():
    [ all_vocab.append(i) for des in new_dict[key] for i in des.split()]

In [ ]:
print("total words appearing : " , len(all_vocab))

total words appearing :  454757


In [ ]:
from collections import Counter

counter = Counter(all_vocab)

In [ ]:
dic_ = dict(counter)

In [ ]:
sorted_dic = sorted(dic_.items(), key = lambda x: x[1], reverse=True)   #List with words and their corresponding frequency

In [ ]:
print(len(sorted_dic))

8426


In [ ]:

for i in range(len(sorted_dic)):print(sorted_dic[i]) 

In [ ]:
threshold_value = 10

d = [(x) for x in sorted_dic if x[1]>threshold_value]

In [ ]:
len(d)

1847

In [ ]:
all_vocab = [x[0] for x in d]                       #Updating all_vocab

In [ ]:
len(all_vocab)

1847

In [ ]:
f = open('new_dict.txt', 'w')
f.write(str(new_dict))
f.close()

In [ ]:
with open('/content/drive/My Drive/AI/Projects/Image captioning/Dataset/FlickrText/New folder/Flickr_8k.trainImages.txt') as f:
  train=f.read()

In [ ]:
print(train)

In [ ]:
train = [e[:-4] for e in train.split('\n')[:-1]]                                #train image ids

In [ ]:
print(train)

In [ ]:
with open('/content/drive/My Drive/AI/Projects/Image captioning/Dataset/FlickrText/New folder/Flickr_8k.testImages.txt')as f:
  test=f.read()

In [ ]:
test = [e[:-4] for e in test.split('\n')[:-1]]                                    #test image ids

In [ ]:
train_descriptions = {}                                                           #dictionary with keys as image ids and values as corresponding captions
for t in train:
    train_descriptions[t] = []
    for cap in new_dict[t]:
        train_descriptions[t].append(cap)

In [ ]:
i=0
for keys,values in train_descriptions.items():
 i=i+1
 
 print(keys)
 print(values)
 if i>50:break

Data Preprocessing- Images

In [ ]:
from keras.applications.resnet50 import ResNet50, preprocess_input

In [ ]:
model = ResNet50(weights = 'imagenet', input_shape = (224,224,3))

102973440/102967424 [==============================] - 1s 0us/step


In [ ]:
model.summary()

Model: "resnet50"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 112, 112, 64) 256         conv1_conv[0][0]                 
___________________________________________________________________________________________

In [ ]:
model_new = Model(inputs = model.input, outputs =  model.layers[-2].output)

In [ ]:
def preprocess_image(img):
    img = image.load_img(img, target_size=(224,224))
    img = image.img_to_array(img)
    img = preprocess_input(img)
    img = np.expand_dims(img, axis = 0)

    return img


In [ ]:
def encode_image(img):
    img = preprocess_image(img)
    fea_vec = model_new.predict(img)
    fea_vec = fea_vec.reshape(fea_vec.shape[1], )
    return fea_vec

In [ ]:
images='/content/drive/My Drive/AI/Projects/Image captioning/Dataset/Flicker8k_Dataset/'

In [ ]:
start = time.time()

encoding_train = {}                #Dictionary with keys as image ids and values as encoding corresponding to the images                        

for ix, img in enumerate(train):
    
    img = images+train[ix]+".jpg"
    
    p = encode_image(img)
    
    encoding_train[ img[len(images):] ] = p
    
    
    if ix%100 == 0:
        print("Encoding image :" + str(ix))
    
print("Time taken in sec - " + time.time() - start)

In [ ]:
start = time.time()

encoding_test = {}

for ix, img in enumerate(test):
    
    img = images+test[ix]+".jpg"
    
    p = encode_image(img)
    
    encoding_test[ img[len(images):] ] = p
    
    
    if ix%100 == 0:
        print("Encoding image :" + str(ix))
    
print("Time taken in sec - " + time.time() - start)

In [ ]:
# saving features to disk

with open("./encoded_train_images.pkl", 'wb') as f:
    pickle.dump(encoding_train, f )

In [ ]:
with open("./encoded_test_images.pkl", 'wb') as f:
    pickle.dump(encoding_test, f )

In [ ]:
with open("/content/drive/My Drive/AI/Projects/Image captioning/Dataset/Pkl/encoded_train_images.pkl", 'rb') as f:
    encoding_train = pickle.load(f)

In [ ]:
with open("/content/drive/My Drive/AI/Projects/Image captioning/Dataset/Pkl/encoded_test_images.pkl", 'rb') as f:
    encoding_test = pickle.load(f)

**Captions**

In [ ]:
word_to_idx = {}
idx_to_word = {}

ix = 1

for e in all_vocab:
    #print(ix,e)
    word_to_idx[e] = ix
    idx_to_word[ix] = e
    ix +=1

In [ ]:
i=0
for keys,values in word_to_idx.items():
 i=i+1
 
 print(keys,values)
 
 if i>50:break

<startseq> 1
<endseq> 2
in 3
the 4
on 5
is 6
and 7
dog 8
with 9
man 10
of 11
two 12
white 13
black 14
boy 15
are 16
woman 17
girl 18
to 19
wearing 20
at 21
people 22
water 23
red 24
young 25
brown 26
an 27
his 28
blue 29
dogs 30
running 31
through 32
playing 33
while 34
shirt 35
down 36
standing 37
ball 38
little 39
grass 40
snow 41
child 42
person 43
jumping 44
over 45
three 46
front 47
sitting 48
holding 49
up 50
field 51


In [ ]:
i=0
for keys,values in idx_to_word.items():
 i=i+1
 
 print(keys,values)
 
 if i>50:break

In [ ]:
for i in range(51):print(all_vocab[i])

In [ ]:
len(all_vocab)

1847

In [ ]:
vocab_size = len(idx_to_word) + 1
print(vocab_size)


1848


In [ ]:
all_caption_len = []

for key in train_descriptions.keys():
    for cap in train_descriptions[key]:
        all_caption_len.append(len(cap.split()))

In [ ]:
print(len(all_caption_len))
print(all_caption_len[:50])

In [ ]:
max_len = max(all_caption_len)
print(max_len) 

35


Generator Function

In [ ]:
def data_generator(train_descriptions, encoding_train, word_to_idx, max_len,  num_photos_per_batch ):
    X1, X2, y = [], [], []
    
    n=0
    cnt = 0
    all_items = list(train_descriptions.keys())
    
    while True:
      n+=1
      # print(cnt)
      key = all_items[cnt]
      desc_list = train_descriptions[key]              
      cnt+=1
      cnt= (cnt%len(all_items))


      photo = encoding_train[key+'.jpg']          #feature vector
            #print(photo.shape)
      for desc in desc_list:                       #desc : iterates through the 5 captions
        seq = [word_to_idx[word] for word in desc.split() if word in word_to_idx]                
                
        for i in range(1, len(seq)): 
            in_seq = seq[0:i]
            out_seq = seq[i]
            
            in_seq = pad_sequences( [in_seq], maxlen=max_len, value= 0, padding='post')[0]
        
            out_seq = to_categorical([out_seq], num_classes=vocab_size)[0]
            
            X1.append(photo)
            X2.append(in_seq)
            y.append(out_seq)

      if n == num_photos_per_batch:
          yield [np.array(X1), np.array(X2)] , np.array(y)
          X1, X2, y = [], [], []
          n = 0         

In [ ]:
for i in data_generator(train_descriptions, encoding_train, word_to_idx, max_len, 3):
    X, y = i
    print(X[0].shape) 
    print(X[1].shape)
    print(y.shape)
    break

(147, 2048)
(147, 35)
(147, 1848)


Word Embedding

In [ ]:
embeddings = {}

with open('/content/drive/My Drive/AI/Projects/Image captioning/Dataset/FlickrText/glove.6B.50d.txt', 'r', encoding='utf-8') as f:
    for line in f:
        values = line.split()
        word = values[0]
        coeffs = np.array(values[1:], dtype="float32")
        
        embeddings[word] = coeffs


In [ ]:
i=0
for keys,values in embeddings.items():
 i=i+1
 
 print(keys,values)
 
 if i>10:break

In [ ]:
print(len(embeddings))

400000


In [ ]:
def getOutputEmbeddings():

    emb_dim = 50
    embedding_matrix_output = np.zeros((vocab_size, emb_dim ))
    
    for word, idx in word_to_idx.items():
        
        emb_vec = embeddings.get(word)
        
        if emb_vec is not None:
            embedding_matrix_output[idx] = emb_vec
            
    return embedding_matrix_output

In [ ]:
embedding_output = getOutputEmbeddings()

In [ ]:
embedding_output.shape

(1848, 50)

In [ ]:
print(embedding_output[5:7])

[[ 3.00449997e-01  2.50059992e-01 -1.66920006e-01  1.92300007e-01
   2.69210003e-02 -7.94859976e-02 -9.13829982e-01 -1.97400004e-01
  -5.34130000e-02 -4.08459991e-01 -2.68440008e-01 -2.82119989e-01
  -5.00000000e-01  1.22100003e-01  3.90300006e-01  1.77970007e-01
  -4.42900002e-01 -4.04780000e-01 -9.50500011e-01 -1.68970004e-01
   7.77930021e-01  3.35249990e-01  3.34600002e-01 -1.75400004e-01
  -1.20169997e-01 -1.78610003e+00  2.92409986e-01  5.59329987e-01
   2.99820006e-02 -3.24169993e-01  3.92969990e+00  1.08800001e-01
  -5.73350012e-01 -1.78420007e-01  4.17480012e-03 -1.63090006e-01
   4.50769991e-01 -1.61229998e-01 -1.73109993e-01 -8.78890008e-02
  -8.90320018e-02  6.20010011e-02 -1.99460000e-01 -3.88630003e-01
  -1.82319999e-01  6.07509986e-02  9.86030027e-02 -7.13099986e-02
   2.30519995e-01 -5.19389987e-01]
 [ 6.18499994e-01  6.42539978e-01 -4.65519994e-01  3.75699997e-01
   7.48380005e-01  5.37389994e-01  2.22390005e-03 -6.05769992e-01
   2.64079988e-01  1.17030002e-01  4.3722

Model Architecture

In [ ]:
input_img_fea = Input(shape=(2048,))
inp_img1 = Dropout(0.3)(input_img_fea)
inp_img2 = Dense(256, activation='relu')(inp_img1)

In [ ]:
print(inp_img2.shape)

(None, 256)


In [ ]:
input_cap = Input(shape=(max_len,))
inp_cap1 = Embedding(input_dim= vocab_size, output_dim=50, mask_zero=True)(input_cap)
#print(inp_cap1)
inp_cap2 = Dropout(0.3)(inp_cap1)
inp_cap3 = LSTM(256)(inp_cap2)

In [ ]:
print(inp_cap3.shape)

(None, 256)


In [ ]:
decoder1 = add([inp_img2, inp_cap3])
print(decoder1.shape)
decoder2 = Dense(256, activation='relu')(decoder1)
output = Dense(vocab_size, activation='softmax')(decoder2)


model = Model(inputs = [input_img_fea, input_cap]  , outputs =  output )

(None, 256)


In [ ]:
model.summary()

Model: "functional_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_4 (InputLayer)            [(None, 35)]         0                                            
__________________________________________________________________________________________________
input_3 (InputLayer)            [(None, 2048)]       0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, 35, 50)       92400       input_4[0][0]                    
__________________________________________________________________________________________________
dropout_1 (Dropout)             (None, 2048)         0           input_3[0][0]                    
_______________________________________________________________________________________

In [ ]:
model.layers[2].set_weights([embedding_output])
model.layers[2].trainable = False

In [ ]:
model.compile(loss="categorical_crossentropy", optimizer='adam') 

In [ ]:
epochs = 10
number_photos_per_batch = 3
steps = len(train_descriptions)//number_photos_per_batch

mytraingen = data_generator(train_descriptions, encoding_train, word_to_idx, max_len, number_photos_per_batch)

model.fit(mytraingen,steps_per_epoch=steps,epochs = epochs)
#model.save("best_model1.h5") 


In [ ]:
model.save( filepath="/content/drive/My Drive/AI/Projects/Image captioning/Model/best_model.h5") 

In [ ]:
#model.save_weights("/content/drive/My Drive/Weights/weights1.h5") 

In [ ]:
epochs = 20
number_photos_per_batch = 3
steps = len(train_descriptions)//number_photos_per_batch

mytraingen = data_generator(train_descriptions, encoding_train, word_to_idx, max_len, number_photos_per_batch)

model.fit(mytraingen,steps_per_epoch=steps,epochs = epochs)
model.save(filepath="/content/drive/My Drive/AI/Projects/Image captioning/Model/best_model1.h5") 

In [ ]:
epochs = 10
number_photos_per_batch = 3
steps = len(train_descriptions)//number_photos_per_batch

mytraingen = data_generator(train_descriptions, encoding_train, word_to_idx, max_len, number_photos_per_batch)

model.fit(mytraingen,steps_per_epoch=steps,epochs = epochs)
#model.save("best_model1.h5") 

In [ ]:
model.save_weights("/content/drive/My Drive/AI/Projects/Image captioning/Weights/weights2.h5") 

In [ ]:
def predict(photo_enc,model):
    in_text = "<startseq>"
    
    for i in range(max_len):
        sequence = [word_to_idx[word] for word in in_text.split() if word in word_to_idx]
        #print(sequence)
        sequence = pad_sequences([sequence], maxlen=max_len, padding='post')
        
        y_pred = model.predict([photo_enc, sequence])
        y_pred = np.argmax(y_pred)
        word = idx_to_word[y_pred]
        
        in_text += " "+word
        
        if word == '<endseq>':
            break
        
        
    final_caption = in_text.split()
    final_caption = final_caption[1:-1]
    final_caption = " ".join(final_caption)
    return final_caption

In [ ]:
#in_text = "<startseq>"
#for i in range(max_len):
        #sequence = [word_to_idx[word] for word in in_text.split() if word in word_to_idx]
        #print(sequence)
        #sequence = pad_sequences([sequence], maxlen=max_len, padding='post')
#print(sequence)        

In [ ]:
#print(word_to_idx['<startseq>'])

In [ ]:
model=load_model('/content/drive/My Drive/AI/Projects/Image captioning/Model/best_model6.h5')
model1=load_model('/content/drive/My Drive/AI/Projects/Image captioning/Model/best_model350.h5')
model2=load_model('/content/drive/My Drive/AI/Projects/Image captioning/Model/best_model7.h5')

In [ ]:
for rn in range(50):
 img_id = list(encoding_test.keys())[rn]

 photo_enc = encoding_test[img_id].reshape((1,2048))
 pred = predict(photo_enc,model1)
 pred1 = predict(photo_enc,model)
 pred2 = predict(photo_enc,model2)

 print('1 '+pred)
 print('2 '+pred1)
 print('3 '+pred2)

 path = images + img_id
 img = plt.imread(path)
 plt.imshow(img)
 plt.show()

In [ ]:
#model=load_model('/content/drive/My Drive/AI/Projects/Image captioning/Model/best_model6.h5')


In [ ]:
#epochs = 5
#number_photos_per_batch = 3
#steps = len(train_descriptions)//number_photos_per_batch

#mytraingen = data_generator(train_descriptions, encoding_train, word_to_idx, max_len, number_photos_per_batch)

#model.fit(mytraingen,steps_per_epoch=steps,epochs = epochs)

In [ ]:
#epochs = 150
#number_photos_per_batch = 3
#steps = len(train_descriptions)//number_photos_per_batch

#mytraingen = data_generator(train_descriptions, encoding_train, word_to_idx, max_len, number_photos_per_batch)

#model.fit(mytraingen,steps_per_epoch=steps,epochs = epochs)
#model.save_weights("/content/drive/My Drive/AI/Projects/Image captioning/Weights/weights"+str(20)+ ".h5")
        
       

In [ ]:
m#odel.save( filepath="/content/drive/My Drive/AI/Projects/Image captioning/Model/best_model350.h5") 

In [ ]:
#sequence = pad_sequences([1], maxlen=35, padding='post')
#y_pred = model.predict([photo_enc, sequence])

In [ ]:
#epochs = 20
#number_photos_per_batch = 3
#steps = len(train_descriptions)//number_photos_per_batch

#mytraingen = data_generator(train_descriptions, encoding_train, word_to_idx, max_len, number_photos_per_batch)

#model.fit(mytraingen,steps_per_epoch=steps,epochs = epochs)

In [ ]:
#model.save("best_model5.h5") 

In [ ]:
#model.save_weights("/content/drive/My Drive/Weights/weights5.h5") 

In [ ]:
#model.load_weights('/content/drive/My Drive/Weights/weights1.h5')

In [ ]:
#model.compile(optimizer='adam',loss='categorical_crossentropy')